# Table of Content
01. Import Libraries
02. Import Data
03. Add Address Column
04. Get Coordinates of Flats from OneMap API
05. Get Missing Coordinates Manually
06. Compile A Complete Flat Coordinates List
07. Add Flat Coordinates to Main Dataframe
08. Export Data

# 01. Import Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import copy
import asyncio     # the library needed for asynchronous operation
import nest_asyncio     # the library that allows nested use of the asyncio event loop, making asyncio compatible with Jupyter notebooks
import aiohttp     # the library that allows asychronous API calls
import time        # time library is used to handle time-related tasks

In [2]:
# Allow nested asyncio calls in Jupyter notebooks
nest_asyncio.apply()

In [3]:
# Adjust setting to allow seeing all the rows in the output of this notebook
pd.options.display.max_rows = None

In [4]:
# Adjust setting to allow seeing all the columns in the output of this notebook
pd.options.display.max_columns = None

# 02. Import Data

In [5]:
# Define the main project folder path
path = r'C:\Users\saich\Desktop\CareerFoundry\Data Immersion\Achievement 6 Advanced Analytics & Dashboard Design\11-2023 HDB Flat Resale Analysis'

In [6]:
flat_resale = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'flat_resale_all (checked).csv'), index_col = 0)

In [7]:
flat_resale.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4


In [8]:
flat_resale.shape

(915371, 12)

# 03. Add Address Column

To get coordinates information from OneMap API, a complete address is needed for accurate search. 

Address = block + street name

In [9]:
flat_resale['address'] = flat_resale['block'] + ' ' + flat_resale['street_name']

In [10]:
flat_resale.head(10)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0,309 ANG MO KIO AVE 1
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1,309 ANG MO KIO AVE 1
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2,309 ANG MO KIO AVE 1
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3,309 ANG MO KIO AVE 1
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4,216 ANG MO KIO AVE 1
5,1990-01-01,ANG MO KIO,3 ROOM,211,ANG MO KIO AVE 3,01 TO 03,67.0,New Generation,1977,46000.0,86,5,211 ANG MO KIO AVE 3
6,1990-01-01,ANG MO KIO,3 ROOM,202,ANG MO KIO AVE 3,07 TO 09,67.0,New Generation,1977,42000.0,86,6,202 ANG MO KIO AVE 3
7,1990-01-01,ANG MO KIO,3 ROOM,235,ANG MO KIO AVE 3,10 TO 12,67.0,New Generation,1977,38000.0,86,7,235 ANG MO KIO AVE 3
8,1990-01-01,ANG MO KIO,3 ROOM,235,ANG MO KIO AVE 3,04 TO 06,67.0,New Generation,1977,40000.0,86,8,235 ANG MO KIO AVE 3
9,1990-01-01,ANG MO KIO,3 ROOM,232,ANG MO KIO AVE 3,01 TO 03,67.0,New Generation,1977,47000.0,86,9,232 ANG MO KIO AVE 3


In [11]:
flat_resale.shape

(915371, 13)

# 04. Get Coordinates of Flats from OneMap API

### 04.1 Extract A List of Unique Flat Addresses

In [12]:
flat_addresses = flat_resale['address'].unique()

In [13]:
flat_addresses

array(['309 ANG MO KIO AVE 1', '216 ANG MO KIO AVE 1',
       '211 ANG MO KIO AVE 3', ..., '673C YISHUN AVE 4',
       '468A YISHUN ST 43', '674B YISHUN AVE 4'], dtype=object)

In [14]:
len(flat_addresses)

9804

### 04.2 Use aiohttp and Rate Limiting to Get Coordinates Data from OneMap API

In [17]:
# The URL that connects with OneMap API
url = 'https://onemap.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1'

# 'start' here indicates the starting time of the retrieval process
# The time.time() function returns the number of seconds passed since epoch (the point where time begins)
start = time.time()

# These 3 lists below would be the outcome we want
flat_address = []
latitude = []
longitude = []

# Define a coroutine with async def
async def get_coordinates(session, semaphore, address, count): 
    
    # Use async with statement to utilize the Semaphore
    async with semaphore: 
        
        # Using the session created, connect with OneMap API with get() function to get the information from the API
        # await is added as we need to wait the call to complete in the event loop and return a response, else it won't do anything
        response = await session.get(url.format(address))

        # Parse the JSON response using .json() method 
        # await is addded since the response is an awaitable object
        data = await response.json()

        if data['found'] > 0:     # If the number of items found is not zero
            lat = data['results'][0]['LATITUDE']
            long = data['results'][0]['LONGITUDE']
        else:                     # If the number of items found is zero
            lat = 'Not found'
            long = 'Not found'
        print('{}) {}, {}, {}'.format(count, address, lat, long))

        # Return the addresses and the coordinates retrieved
        return address, lat, long
    
# Since inside the main() function involve coroutines, add 'async' in front of 'def'
# This tells Python that this function can do asynchronous stuff
async def main(): 
    
    count = 1
    tasks = []
    # Initialize a Semaphore 
    # If Semaphore value is 20, max 20 API calls would be made at the same time. Only 20 calls would be in queue. 
    semaphore = asyncio.Semaphore(20)
    
    # A client session is created to allow the underlying TCP connection to be reused for all requests
    # Using context manager 'with' statement will close the client session automatically when all requests are done
    # Since inside this session involve coroutines, add 'async' in front of 'with'
    async with aiohttp.ClientSession() as session: 
        
        for address in flat_addresses: 
            
            # For each address, create a task of running get_coordinates coroutine using asyncio.create_task()
            # A task will be executed asap whenever there is a stall (await) ongoing
            # When the task is stalled, it will go back to the main caller and run other stuff
            # When the main caller stuff is stalled, it will come back to this task and run the remaining stuff
            # So in our case, while waiting for a response to return, it would start other tasks (making other API calls)
            # Remember to pass the session into get_coordinates coroutine so that a single session is used for all requests
            # Create a list of tasks using .append method
            tasks.append(asyncio.create_task(get_coordinates(session, semaphore, address, count)))
            count += 1
        
        # After creating a list of tasks, gather all the tasks created using asyncio.gather()
        # Since the tasks is inside a list, use *tasks to unpack the tasks inside the list
        # await is added to execute all the task and wait them to finish
        # Store the returned values in variable 'results'
        results = await asyncio.gather(*tasks)
    
    fail = 0
    # The result of asyncio.gather(*tasks) is a list of tuples
    # Each tuple contains a address, its latitude and longitude returned by get_coordinates coroutine
    for result_tuple in results: 
        
        address, lat, long = result_tuple
        flat_address.append(address)
        latitude.append(lat)
        longitude.append(long)
        
        if lat == 'Not found': 
            fail += 1
    
    print('A total of {} failed search'.format(fail))

# Run the event loop (the loop of an asynchronous operation) by running main()
asyncio.run(main())

# 'end' here indicates the ending time of the retrieval process
end = time.time()
total_time = end - start
print('Total {} seconds used'.format(total_time))

20) 345 ANG MO KIO AVE 3, 1.36748702509964, 103.849651723914
3) 211 ANG MO KIO AVE 3, 1.369196965617, 103.841666636086
4) 202 ANG MO KIO AVE 3, 1.36844644010937, 103.844516260527
15) 329 ANG MO KIO AVE 3, 1.3665990830055, 103.846874423696
19) 343 ANG MO KIO AVE 3, 1.36837475142719, 103.849151106677
11) 320 ANG MO KIO AVE 1, 1.3655924562134, 103.848386744168
8) 220 ANG MO KIO AVE 1, 1.36558833593063, 103.840518883254
22) 121 ANG MO KIO AVE 3, 1.36970563478325, 103.843515981311
24) 130 ANG MO KIO AVE 3, 1.37039791675237, 103.841878840032
12) 252 ANG MO KIO AVE 4, Not found, Not found
26) 127 ANG MO KIO AVE 3, 1.37122322102767, 103.843593321222
21) 346 ANG MO KIO AVE 3, 1.36709586875167, 103.84966230288
25) 128 ANG MO KIO AVE 3, 1.37083202800001, 103.843546990613
23) 129 ANG MO KIO AVE 3, 1.36989498895756, 103.842059049722
27) 126 ANG MO KIO AVE 3, 1.37164990020956, 103.843644176374
28) 403 ANG MO KIO AVE 10, 1.36126901451361, 103.854642365822
32) 418 ANG MO KIO AVE 10, 1.3649460137791, 1

151) 87 BEDOK NTH ST 4, 1.33250261426447, 103.938587125421
155) 401 BEDOK NTH AVE 3, 1.33098173461878, 103.933604621756
158) 418 BEDOK NTH AVE 2, 1.32856421177174, 103.930036524233
157) 416 BEDOK NTH AVE 2, 1.32840382815915, 103.930684870347
156) 425 BEDOK NTH RD, 1.32936960154503, 103.929067536324
159) 424 BEDOK NTH AVE 1, 1.32842000962637, 103.927482677521
161) 136 BEDOK NTH AVE 3, 1.32815498828204, 103.93522082327
160) 419 BEDOK NTH ST 1, 1.32697958373998, 103.92813074959
162) 529 BEDOK NTH ST 3, 1.33426946144275, 103.926542284602
163) 510 BEDOK NTH ST 3, 1.33212313291916, 103.93105149637
164) 525 BEDOK NTH ST 3, 1.33460113941761, 103.929445271638
166) 218 BEDOK NTH ST 1, 1.32749295541456, 103.934249434214
165) 202 BEDOK NTH ST 1, 1.32565628436985, 103.927990975389
167) 213 BEDOK NTH ST 1, 1.32633302564898, 103.932636597017
168) 532 BEDOK NTH ST 3, 1.33419173917878, 103.925386600613
170) 536 BEDOK NTH ST 3, 1.3335411894605, 103.923190774432
169) 533 BEDOK NTH ST 3, 1.33358016248275,

306) 44 TELOK BLANGAH DR, 1.27140883340015, 103.810887524155
303) 33 TAMAN HO SWEE, Not found, Not found
307) 46 TELOK BLANGAH DR, 1.27169061066277, 103.809852257535
309) 110 DEPOT RD, 1.28138753916039, 103.809362548941
308) 103 DEPOT RD, 1.28131574987693, 103.807861854014
312) 133 JLN BT MERAH, 1.27955093259631, 103.82740093837
314) 77 INDUS RD, 1.29113483646901, 103.826764453591
311) 132 JLN BT MERAH, 1.27873808127932, 103.828304288385
313) 96 HENDERSON RD, 1.28754332229134, 103.821335072268
310) 42 TELOK BLANGAH RISE, 1.2709187086214, 103.822685388438
315) 78 INDUS RD, 1.29130442807028, 103.827447793798
317) 130 BT MERAH VIEW, 1.28458712798457, 103.824360319186
321) 105 HENDERSON CRES, 1.29021420176947, 103.822356844711
316) 128 BT MERAH VIEW, 1.28264675199072, 103.823327111496
323) 8 TELOK BLANGAH CRES, 1.27779539068994, 103.819341303524
319) 114 BT MERAH VIEW, Not found, Not found
318) 119 BT MERAH VIEW, 1.28342247684697, 103.821068227228
320) 5 TELOK BLANGAH CRES, 1.2787240290542

460) 315 CLEMENTI AVE 4, 1.31847036567508, 103.765419463873
461) 311 CLEMENTI AVE 4, 1.31935456047475, 103.7651093416
462) 301 CLEMENTI AVE 4, 1.32217649495476, 103.764912144396
463) 609 CLEMENTI WEST ST 1, 1.30409242562108, 103.765522148085
464) 729 CLEMENTI WEST ST 2, 1.30492724290307, 103.763519401903
466) 371 CLEMENTI AVE 4, 1.32057465713359, 103.768211873157
465) 707 CLEMENTI WEST ST 2, 1.30628748271347, 103.762394965532
467) 205 CLEMENTI AVE 6, 1.3210599628039, 103.762434749863
469) 415 C'WEALTH AVE WEST, 1.31098804719696, 103.771585053355
468) 414 C'WEALTH AVE WEST, 1.31158107279643, 103.771625045985
470) 413 C'WEALTH AVE WEST, 1.31134730442536, 103.771097378247
471) 336 CLEMENTI AVE 2, 1.31667943214563, 103.769231150532
472) 337 CLEMENTI AVE 2, 1.31605567747075, 103.769112678697
474) 723 CLEMENTI WEST ST 2, 1.30310124001611, 103.763413053663
476) 103 CLEMENTI ST 14, 1.32258854750628, 103.769176205799
473) 717 CLEMENTI WEST ST 2, 1.30323440870789, 103.762464847394
477) 204 CLEME

613) 408 PANDAN GDNS, 1.31820397685545, 103.74881622665
614) 218 JURONG EAST ST 21, 1.34033241177285, 103.737928007136
612) 251 JURONG EAST ST 24, 1.34316585637535, 103.73967649195
615) 322 JURONG EAST ST 31, 1.34803793215272, 103.729165542694
610) 316 JURONG EAST ST 32, 1.34600652715087, 103.732079814028
611) 232 JURONG EAST ST 21, 1.33982600141552, 103.740822151959
616) 227 JURONG EAST ST 21, 1.34182066937117, 103.736087793751
618) 103 JURONG EAST ST 13, 1.33668715015786, 103.738059200285
617) 109 JURONG EAST ST 13, 1.3383617052828, 103.735911158508
619) 240 JURONG EAST ST 21, 1.34106751096494, 103.741573199896
620) 258 JURONG EAST ST 24, 1.34378634902456, 103.738635757888
622) 324 JURONG EAST ST 31, 1.3486489933486, 103.729868988347
621) 209 JURONG EAST ST 21, 1.33782546503407, 103.740093358961
625) 403 PANDAN GDNS, 1.32032964075934, 103.74939013157
623) 331 JURONG EAST AVE 1, 1.35082650419629, 103.73042009299
624) 405 PANDAN GDNS, 1.31923420667707, 103.749654325298
626) 404 PANDAN 

...

9114) 672D EDGEFIELD PLAINS, 1.40759031189113, 103.910923572748
9111) 527C PASIR RIS ST 51, 1.36937329925111, 103.947970231713
9115) 259A PUNGGOL FIELD, 1.40382339950391, 103.895547769394
9122) 890B TAMPINES AVE 1, 1.34690555756363, 103.933409216927
9116) 259C PUNGGOL FIELD, 1.40451590916739, 103.895664857315
9120) 336C ANCHORVALE CRES, 1.39774373171815, 103.88858318745
9123) 342A YISHUN RING RD, 1.43059711114855, 103.84446482342
9126) 505D YISHUN ST 51, 1.41883284175427, 103.844225827265
9125) 505B YISHUN ST 51, 1.41924161070524, 103.844941652639
9117) 667A PUNGGOL DR, 1.40260030998864, 103.913971396177
9121) 330A ANCHORVALE ST, 1.39662723563942, 103.889865427006
9118) 260A PUNGGOL WAY, 1.40479713792879, 103.896616087907
9124) 504D YISHUN ST 51, 1.41833362541375, 103.843721655882
9127) 650 ANG MO KIO ST 61, 1.37769118043958, 103.84274515222
9119) 336A ANCHORVALE CRES, 1.39754408575022, 103.88932509626
9129) 488A CHOA CHU KANG AVE 5, 1.37705950007098, 103.738981398036
9128) 634B SENJA 

9261) 57 TEBAN GDNS RD, 1.32254226544254, 103.737560699176
9266) 868B TAMPINES AVE 8, 1.35592432083149, 103.932912367489
9265) 87 DAWSON RD, 1.2957304268451, 103.809759937443
9262) 256C SUMANG WALK, 1.40540203681824, 103.895910481206
9263) 256D SUMANG WALK, 1.40494870154993, 103.896007686194
9267) 440B CLEMENTI AVE 3, 1.31623936538205, 103.763566375801
9271) 530A PASIR RIS DR 1, 1.37114053931458, 103.949850548422
9270) 530B PASIR RIS DR 1, 1.37168953845762, 103.949502101347
9254) 871C TAMPINES ST 86, 1.35670900289357, 103.931385349285
9272) 668A EDGEFIELD PLAINS, 1.40501880052452, 103.912411867708
9268) 376C HOUGANG ST 32, 1.36406322514465, 103.893679750644
9269) 8A UPP BOON KENG RD, 1.31291757072948, 103.869743094328
9276) 294A COMPASSVALE CRES, 1.39606977416428, 103.895933432889
9274) 470B FERNVALE LINK, 1.39687117865647, 103.879859508141
9264) 678A PUNGGOL DR, 1.40566213236302, 103.908718985826
9273) 88 DAWSON RD, 1.29579395038247, 103.809264765809
9275) 473A FERNVALE ST, 1.39785974

9399) 116B JLN TENTERAM, 1.32754908001444, 103.860807411661
9385) 312C CLEMENTI AVE 4, 1.31952267914453, 103.764025116013
9398) 698A JURONG WEST CTRL 3, 1.341091607667, 103.708023659485
9402) 662B EDGEDALE PLAINS, 1.40172153675655, 103.918690999509
9397) 817B KEAT HONG LINK, 1.37708774602652, 103.748768050663
9396) 805C KEAT HONG CL, 1.37268736243732, 103.743618216079
9389) 314B PUNGGOL WAY, 1.40789778716346, 103.897370752408
9401) 662C EDGEDALE PLAINS, 1.40139181635046, 103.918513893557
9405) 278B COMPASSVALE BOW, 1.38235083780012, 103.890641438035
9404) 286C COMPASSVALE CRES, 1.39811786720081, 103.896423686037
9407) 194B BT BATOK WEST AVE 6, 1.34513283900191, 103.744861953262
9408) 194A BT BATOK WEST AVE 6, 1.34468050128788, 103.7455138661
9403) 314A PUNGGOL WAY, 1.40826187839775, 103.898017940475
9410) 812C CHOA CHU KANG AVE 7, 1.3727609833656, 103.745917783867
9406) 717A WOODLANDS DR 70, 1.44053874571164, 103.798669114626
9412) 802A KEAT HONG CL, 1.37451584184376, 103.74347432386
9

9534) 92A TELOK BLANGAH ST 31, 1.27653169514471, 103.807514165495
9531) 165 BT MERAH CTRL, 1.2829569880635, 103.816934790553
9538) 561A JURONG WEST ST 42, 1.354294109695, 103.719755141086
9535) 90B TELOK BLANGAH ST 31, 1.27728367741324, 103.806916176715
9537) 217B BOON LAY AVE, 1.34972661567521, 103.710830462207
9536) 218B BOON LAY AVE, 1.35032707359938, 103.710888928549
9539) 218D BOON LAY AVE, 1.35085552032825, 103.712083394384
9540) 561B JURONG WEST ST 42, 1.35477102717501, 103.71967262747
9541) 217A BOON LAY AVE, 1.34975768459224, 103.711296415279
9543) 216B BOON LAY AVE, 1.35045319598788, 103.712256991806
9542) 560A JURONG WEST ST 42, 1.3533579932237, 103.719829055981
9546) 165B YUNG KUANG RD, 1.33064472260389, 103.722321863113
9544) 218A BOON LAY AVE, 1.34996858706692, 103.710539626132
9547) 165A YUNG KUANG RD, 1.33072542257431, 103.721913762657
9548) 166B YUNG KUANG RD, 1.3312465772439, 103.722255844894
9545) 164A YUNG KUANG RD, 1.33018740882956, 103.721875506696
9550) 78 LOR LI

9682) 186A BEDOK NTH ST 4, 1.32991427421905, 103.940310513079
9680) 187A BEDOK NTH ST 4, 1.33087094745916, 103.940210775064
9687) 441A BT BATOK WEST AVE 8, 1.35610548541132, 103.739002965384
9681) 187B BEDOK NTH ST 4, 1.33038146607469, 103.940088401203
9686) 440A BT BATOK WEST AVE 8, 1.35522654404615, 103.738772439227
9691) 436A BT BATOK WEST AVE 5, 1.35912883183507, 103.739978327829
9688) 448A BT BATOK WEST AVE 9, 1.35286511774512, 103.740438584574
9689) 448B BT BATOK WEST AVE 9, 1.35314900034194, 103.740846555722
9690) 377 BT BATOK ST 31, 1.35942058239166, 103.750290022372
9693) 292B BT BATOK EAST AVE 6, 1.34477953186313, 103.753961675829
9692) 441C BT BATOK WEST AVE 8, 1.35556816773842, 103.738704484722
9695) 435C BT BATOK WEST AVE 5, 1.35830193962809, 103.739753503555
9696) 451B BT BATOK WEST AVE 6, 1.35248381356888, 103.743414725495
9694) 293B BT BATOK ST 21, 1.34594925522117, 103.754248349899
9697) 450B BT BATOK WEST AVE 6, 1.35235753004524, 103.744395918688
9702) 445A BT BATOK W

A total of 129 seconds are used. <br>
There are 137 flat addresses failed to get their coordinates.

In [18]:
# Check the data stored in the 'flat_address' list
flat_address

['309 ANG MO KIO AVE 1',
 '216 ANG MO KIO AVE 1',
 '211 ANG MO KIO AVE 3',
 '202 ANG MO KIO AVE 3',
 '235 ANG MO KIO AVE 3',
 '232 ANG MO KIO AVE 3',
 '308 ANG MO KIO AVE 1',
 '220 ANG MO KIO AVE 1',
 '219 ANG MO KIO AVE 1',
 '247 ANG MO KIO AVE 3',
 '320 ANG MO KIO AVE 1',
 '252 ANG MO KIO AVE 4',
 '223 ANG MO KIO AVE 1',
 '230 ANG MO KIO AVE 3',
 '329 ANG MO KIO AVE 3',
 '313 ANG MO KIO AVE 3',
 '117 ANG MO KIO AVE 4',
 '110 ANG MO KIO AVE 4',
 '343 ANG MO KIO AVE 3',
 '345 ANG MO KIO AVE 3',
 '346 ANG MO KIO AVE 3',
 '121 ANG MO KIO AVE 3',
 '129 ANG MO KIO AVE 3',
 '130 ANG MO KIO AVE 3',
 '128 ANG MO KIO AVE 3',
 '127 ANG MO KIO AVE 3',
 '126 ANG MO KIO AVE 3',
 '403 ANG MO KIO AVE 10',
 '404 ANG MO KIO AVE 10',
 '405 ANG MO KIO AVE 10',
 '417 ANG MO KIO AVE 10',
 '418 ANG MO KIO AVE 10',
 '419 ANG MO KIO AVE 10',
 '441 ANG MO KIO AVE 10',
 '442 ANG MO KIO AVE 10',
 '443 ANG MO KIO AVE 10',
 '444 ANG MO KIO AVE 10',
 '450 ANG MO KIO AVE 10',
 '435 ANG MO KIO AVE 10',
 '433 ANG MO 

In [19]:
len(flat_address)

9804

In [20]:
# Check the data stored in the 'latitude' list
latitude

['Not found',
 '1.36619678831054',
 '1.369196965617',
 '1.36844644010937',
 '1.36682360872342',
 '1.36834606813391',
 '1.36569665029696',
 '1.36558833593063',
 '1.36598198004505',
 'Not found',
 '1.3655924562134',
 'Not found',
 '1.36604808445916',
 '1.36885445228086',
 '1.3665990830055',
 '1.36664972640598',
 '1.3737170825368',
 '1.37045937010888',
 '1.36837475142719',
 '1.36748702509964',
 '1.36709586875167',
 '1.36970563478325',
 '1.36989498895756',
 '1.37039791675237',
 '1.37083202800001',
 '1.37122322102767',
 '1.37164990020956',
 '1.36126901451361',
 '1.36167767060852',
 '1.36157912717139',
 '1.36495953274783',
 '1.3649460137791',
 '1.36534669720177',
 '1.36605027829032',
 '1.36569241524537',
 '1.36604388871674',
 '1.36660045321147',
 '1.36848406233048',
 '1.36770367670019',
 '1.36807652285135',
 '1.3678915069194',
 '1.36860660305158',
 '1.36915358068729',
 '1.36581960620891',
 '1.36346600647245',
 '1.36275784702216',
 '1.3697435855576',
 '1.37030352965928',
 'Not found',
 '1.366

In [21]:
len(latitude)

9804

In [22]:
# Check the data stored in the 'latitude' list
longitude

['Not found',
 '103.841505011903',
 '103.841666636086',
 '103.844516260527',
 '103.83649123351',
 '103.837196046409',
 '103.844347269095',
 '103.840518883254',
 '103.840654039612',
 'Not found',
 '103.848386744168',
 'Not found',
 '103.838122716883',
 '103.837896397823',
 '103.846874423696',
 '103.850133665442',
 '103.835610469339',
 '103.837419259859',
 '103.849151106677',
 '103.849651723914',
 '103.84966230288',
 '103.843515981311',
 '103.842059049722',
 '103.841878840032',
 '103.843546990613',
 '103.843593321222',
 '103.843644176374',
 '103.854642365822',
 '103.854726492538',
 '103.853805386957',
 '103.855197097728',
 '103.854043379775',
 '103.85415182597',
 '103.854168309853',
 '103.855097870221',
 '103.855368968681',
 '103.855578790668',
 '103.855468149019',
 '103.854214321442',
 '103.854353042615',
 '103.85345502653',
 '103.850808805402',
 '103.851746834886',
 '103.857061597256',
 '103.856702918462',
 '103.858015323667',
 '103.856023138277',
 '103.853455898206',
 'Not found',
 '1

In [23]:
len(longitude)

9804

From the result, there are a total of 137 fail counts. This means that these addresses have no coordinates obtained from API.

This issue will be handled in section 5 below.

### 04.3 Combine the list of hdb addresses, latitude and longitude into a new dataframe

In [24]:
flat_coordinates_onemap = pd.DataFrame({'address' : flat_addresses, 
                                        'latitude' : latitude, 
                                        'longitude' : longitude})

In [25]:
flat_coordinates_onemap.head(10)

,address,latitude,longitude
0,309 ANG MO KIO AVE 1,Not found,Not found
1,216 ANG MO KIO AVE 1,1.36619678831054,103.841505011903
2,211 ANG MO KIO AVE 3,1.369196965617,103.841666636086
3,202 ANG MO KIO AVE 3,1.36844644010937,103.844516260527
4,235 ANG MO KIO AVE 3,1.36682360872342,103.83649123351
5,232 ANG MO KIO AVE 3,1.36834606813391,103.837196046409
6,308 ANG MO KIO AVE 1,1.36569665029696,103.844347269095
7,220 ANG MO KIO AVE 1,1.36558833593063,103.840518883254
8,219 ANG MO KIO AVE 1,1.36598198004505,103.840654039612
9,247 ANG MO KIO AVE 3,Not found,Not found


In [26]:
flat_coordinates_onemap.tail(10)

,address,latitude,longitude
9794,462A YISHUN AVE 6,1.42763749432623,103.850122944006
9795,462C YISHUN AVE 6,1.42756750243707,103.850898727139
9796,462B YISHUN AVE 6,1.42713422609837,103.850234948425
9797,462D YISHUN AVE 6,1.42803884912514,103.850959508396
9798,676A YISHUN RING RD,1.42145242328288,103.84332790779
9799,469A YISHUN ST 43,1.42469407414992,103.852385726656
9800,469B YISHUN ST 43,1.42438879653642,103.852083098516
9801,673C YISHUN AVE 4,1.42104894176938,103.842206525185
9802,468A YISHUN ST 43,1.42532389372284,103.851843638346
9803,674B YISHUN AVE 4,1.42107254833692,103.843117058752


In [27]:
flat_coordinates_onemap.shape

(9804, 3)

In [28]:
# Export 'flat_address_coordinates_onemap' to Prepared Data folder 
flat_coordinates_onemap.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'flat_coordinates_onemap.csv'))

# 05. Get Missing Coordinates Manually

As mentioned earlier, there are 137 addresses failed to get their coordinates from OneMap API. This might due to the demolition of the blocks through the Selective En bloc Redevelopment Scheme (SERS). For this reason, we are going to obtain their coordinates manually from Google Map and asign to them. We will try to find the postal codes of these addresses from Google and apply them on Google Map. If the postal code is not available and Google Map cannot recognize the address, we will choose the coordinates of the nearest block based on block number. This would not be accurate, but it at least gives us an idea on where were these flats located roughly. <br>
https://www.hdb.gov.sg/residential/living-in-an-hdb-flat/sers-and-upgrading-programmes/sers/sers-projects/completed-sers-projects <br>
https://www.hdb.gov.sg/residential/living-in-an-hdb-flat/sers-and-upgrading-programmes/sers/sers-projects/in-progress-sers-projects

In [29]:
flat_coordinates_onemap.loc[flat_coordinates_onemap['latitude'] == 'Not found']['address']

0           309 ANG MO KIO AVE 1
9           247 ANG MO KIO AVE 3
11          252 ANG MO KIO AVE 4
48         455 ANG MO KIO AVE 10
262               4 HILLVIEW AVE
263               6 HILLVIEW AVE
264               2 HILLVIEW AVE
302             33 TAMAN HO SWEE
318            114 BT MERAH VIEW
340             18 KG BAHRU HILL
341                20 REDHILL CL
342                13 REDHILL CL
343                19 REDHILL CL
346               29 HAVELOCK RD
360            111 BT MERAH VIEW
384                 22 OUTRAM PK
385                  12 SHORT ST
386                  13 SHORT ST
418           323 CLEMENTI AVE 5
419           322 CLEMENTI AVE 5
421           314 CLEMENTI AVE 4
422           313 CLEMENTI AVE 4
429           401 CLEMENTI AVE 1
486             1 JLN PASAR BARU
494                   59 SIMS DR
584              3 TEBAN GDNS RD
585              5 TEBAN GDNS RD
586             10 TEBAN GDNS RD
631              8 YUNG KUANG RD
632             10 YUNG KUANG RD
637       

<b>Below are the coordinates manually input to the 137 addresses above:</b> <br>
Note: The address inside the brackets in each line is used in searching on Google Map. <br>

309 ANG MO KIO AVE 1 - 1.3655169889847343, 103.84396819147089 (308A Ang Mo Kio Ave 1) <br>
247 ANG MO KIO AVE 3 - 1.3682216217409664, 103.8336527877963 (Singapore 560247) <br>
252 ANG MO KIO AVE 4 - 1.369342222650695, 103.83422256323288 (Singapore 560252) <br>
455 ANG MO KIO AVE 10 - 1.3688405731166766, 103.85703511741559 (455 Ang Mo Kio Ave 10) <br>
4 HILLVIEW AVE - 1.3664368833135392, 103.76222396965102 (4 Hillview Ave) <br>
6 HILLVIEW AVE - 1.3662133719859624, 103.76245672057786 (6 Hillview Ave) <br>
2 HILLVIEW AVE - 1.3662026462000616, 103.76239771197721 (2 Hillview Ave) <br>
33 TAMAN HO SWEE - 1.2883291797280607, 103.83174038533298 (Singapore 161033) <br>
114 BT MERAH VIEW - 1.2863867107325693, 103.82228529207782 (Singapore 150114) <br>
18 KG BAHRU HILL - 1.2765777783976922, 103.82997665649788 (Singapore 160018) <br>
20 REDHILL CL - 1.286801643801765, 103.8172859626269 (Singapore 151020) <br>
13 REDHILL CL - 1.2854999115107872, 103.81685556323275 (Singapore 150013) <br>
19 REDHILL CL - 1.2860127029205133, 103.81808404944239 (Singapore 151019) <br>
29 HAVELOCK RD - 1.2891210904821562, 103.83081797431886 (Singapore 160029) <br>
111 BT MERAH VIEW - 1.2862548222415184, 103.82365165648943 (Singapore 150111) <br>
22 OUTRAM PK - 1.2828103563742577, 103.83858979653401 (22 Outram Park) <br>
12 SHORT ST - 1.3028789962902574, 103.85168325166448 (LASALLE College of the Arts) <br>
13 SHORT ST - 1.3028789962902574, 103.85168325166448 (LASALLE College of the Arts) <br>
323 CLEMENTI AVE 5 - 1.3158108171887475, 103.76703436726766 (Singapore 120323) <br>
322 CLEMENTI AVE 5 - 1.316236495800941, 103.766435881021 (Singapore 120322) <br>
314 CLEMENTI AVE 4 - 1.318422518418761, 103.76538188859801 (315 Clementi Avenue 4) <br>
313 CLEMENTI AVE 4 - 1.3190912991344506, 103.76393874736654 (312 Clementi Avenue 4) <br>
401 CLEMENTI AVE 1 - 1.3069446646911946, 103.7681116098622 (Singapore 120401) <br>
1 JLN PASAR BARU - 1.3168125336950494, 103.89820326965021 (Borong Pasar Geylang) <br>
59 SIMS DR - 1.318784652990415, 103.88008490137942 (59 Sims Dr) <br>
3 TEBAN GDNS RD - 1.3217519049381448, 103.74482344240235 (Singapore 600003) <br>
5 TEBAN GDNS RD - 1.3222065990812415, 103.74437327124912 (Singapore 600005) <br>
10 TEBAN GDNS RD - 1.3221922715778167, 103.74263178429575 (Singapore 600010) <br>
8 YUNG KUANG RD - 1.3308797755065527, 103.72239344561419 (Singapore 610008) <br>
10 YUNG KUANG RD - 1.3314581815192128, 103.72195265649171 (Singapore 610010) <br>
145 HU CHING RD - 1.3291068802365438, 103.72334764611611 (Hu Ching Rd) <br>
167 BOON LAY DR - 1.3466973291302466, 103.70618152830032 (Singapore 640167) <br>
168 BOON LAY DR - 1.3469730247979022, 103.70618165648999 (Singapore 640168) <br>
169 BOON LAY DR - 1.3476450724362483, 103.70740006753553 (Singapore 640169) <br>
170 BOON LAY DR - 1.3473275875247912, 103.70730583440012 (Singapore 640170) <br>
172 BOON LAY DR - 1.3465525990524971, 103.7077945157944 (Singapore 640172) <br>
220 BOON LAY AVE - 1.3503128744193946, 103.71196371374799 (Singapore 640220) <br>
20 UPP BOON KENG RD - 1.3142877319758566, 103.87376078200853 (Singapore 380020) <br>
53 TANGLIN HALT RD - 1.300873366742979, 103.79713301931747 (Singapore 142053) <br>
54 TANGLIN HALT RD - 1.3008700727020481, 103.79715160986777 (Singapore 142054) <br>
96 MARGARET DR - 1.299856478092569, 103.8034562223237 (96 Margaret Drive) <br>
24 DOVER CRES - 1.3064596266116126, 103.78280346914953 (Singapore 130024) <br>
173 STIRLING RD - 1.2953599380076886, 103.8045944996247 (173 Stirling Rd) <br>
172 STIRLING RD - 1.295394228129917, 103.80458914378356 (172 Stirling Rd) <br>
52 TANGLIN HALT RD - 1.3008821395274324, 103.79715697427598 (Singapore 142052) <br>
74 C'WEALTH DR - 1.300989084076144, 103.79604097428599 (Singapore 140074) <br>
77 C'WEALTH DR - 1.301470273139427, 103.7957675632403 (Singapore 140077) <br>
79 C'WEALTH DR - 1.301545391133773, 103.79513363318132 (Singapore 140079) <br>
78 C'WEALTH DR - 1.301688435674063, 103.79564629207681 (Singapore 140078) <br>
36 DOVER RD - 1.3000258656886836, 103.78270983437956 (Singapore 130036) <br>
5 SELETAR WEST FARMWAY 6 - 1.396071149317463, 103.86977533869396 (Singapore 798044) <br>
166 LOR 1 TOA PAYOH - 1.3340940263210606, 103.84329745877793 (166 Lor 1 Toa Payoh) <br>
30 LOR 5 TOA PAYOH - 1.33362336095645, 103.854453292076 (Singapore 310030) <br>
1A WOODLANDS CTR RD - 1.4417712984690574, 103.7708907963624 (Singapore 731001) <br>
248 ANG MO KIO AVE 2 - 1.3684988072760422, 103.83237170311509 (Singapore 560248) <br>
454 ANG MO KIO AVE 10 - 1.367897539496139, 103.85793421208811 (Singapore 560454) <br>
83 BEDOK NTH RD - 1.3295455850920797, 103.94026502243322 (Block 82 HDB Bedok North) <br>
31 TAMAN HO SWEE - 1.2887334855943655, 103.83179834072726 (Singapore 164031) <br>
107 DEPOT RD - 1.2815963740997012, 103.80972468295792 (107 Depot Rd) <br>
1 REDHILL CL - 1.2843276927340275, 103.81784602393202 (Singapore 151001) <br>
407 CLEMENTI AVE 1 - 1.3104812154959264, 103.7678115166119 (Singapore 120407) <br>
409 CLEMENTI AVE 1 - 1.3125623724855453, 103.76821794848296 (Singapore 120409) <br>
403 CLEMENTI AVE 1 - 1.3085229540133465, 103.76707350986686 (Singapore 120403) <br>
402 CLEMENTI AVE 1 - 1.307789954116844, 103.76742360979296 (Singapore 120402) <br>
54 SIMS DR - 1.3185404955995619, 103.87880660995062 (Singapore 380054) <br>
62 SIMS DR - 1.3192970325413753, 103.87667188102921 (Singapore 380062) <br>
14 TEBAN GDNS RD - 1.3229017147778237, 103.7408389667023 (Field of Dreams) <br>
5 YUNG PING RD - 1.330170359464526, 103.72186442335509 (Singapore 610005) <br>
6 YUNG PING RD - 1.3303038488929029, 103.72191440431149 (Singapore 610006) <br>
142 HU CHING RD - 1.3291068802365438, 103.72334764611611 (Hu Ching Rd) <br>
171 BOON LAY DR - 1.3467866242790758, 103.70778012524903 (Singapore 640171) <br>
6 UPP BOON KENG RD - 1.3139261114966696, 103.87084409287942 (6 Upper Boon Keng Rd) <br>
76 C'WEALTH DR - 1.3008429101601142, 103.79550129207685 (Singapore 140076) <br>
316 ANG MO KIO AVE 1 - 1.3647397126672844, 103.84664905288479 (316 Ang Mo Kio Ave 1) <br>
110 BT MERAH VIEW - 1.2874444027864156, 103.82270333870572 (Singapore 150110) <br>
22 KG BAHRU HILL - 1.2769419875618853, 103.82935836963716 (22 Silat Rd) <br>
11 SHORT ST - 1.3028789962902574, 103.85168325166448 (LASALLE College of the Arts) <br>
7 TEBAN GDNS RD - 1.321646610152141, 103.74400042304818 (Singapore 600007) <br>
13 TEBAN GDNS RD - 1.3229017147778237, 103.7408389667023 (Field of Dreams) <br>
3 LIM CHU KANG RD - 1.4213081090994577, 103.70420781623427 (Singapore 710003) <br>
10 GHIM MOH RD - 1.3105030133920044, 103.78988852041667 (Singapore 270010) <br>
27A C'WEALTH AVE - 1.2953844530213265, 103.80603633870484 (Singapore 140027) <br>
80 C'WEALTH DR - 1.3019442285514258, 103.79552070312464 (Singapore 140080) <br>
7 HILLVIEW AVE - 1.3658780401799882, 103.76391732283415 (7 Hillview Avenue) <br>
8 HILLVIEW AVE - 1.3663688948516357, 103.7624084363247 (8 Hillview Avenue) <br>
32 LOWER DELTA RD - 1.2889645742650595, 103.82611370994 (48 Lower Delta Rd) <br>
88 ZION RD - 1.2917847509271556, 103.83293731237836 (Singapore 160088) <br>
90 ZION RD - 1.2910245717815583, 103.83369881773812 (Singapore 160090) <br>
8 REDHILL CL - 1.2851824369578624, 103.8167986331832 (Singapore 151008) <br>
3 REDHILL CL - 1.2845610703287467, 103.81669167005137 (Singapore 151003) <br>
51 NILE RD - 1.2797473055291857, 103.84265846047273 (51 Neil Road) <br>
54 NILE RD - 1.2799488141180526, 103.84260959952063 (54 Neil Road) <br>
24 JLN MEMBINA BARAT - 1.2832811287032757, 103.82653184062683 (Singapore 161024) <br>
6 TEBAN GDNS RD - 1.3212062433536578, 103.74401182792772 (Singapore 600006) <br>
35 DOVER RD - 1.299156324478221, 103.78341426876047 (Singapore 130035) <br>
5 BUANGKOK STH FARMWAY 1 - 1.3845278423529006, 103.8985240373611 (Compassvale View) <br>
1 SELETAR WEST FARMWAY 6 - 1.3946329814447982, 103.86998106753063 (Singapore 798042) <br>
2 SELETAR WEST FARMWAY 6 - 1.3947641672830142, 103.87078124543937 (Singapore 798043) <br>
2A WOODLANDS CTR RD - 1.4408120781471487, 103.77085024543479 (Singapore 731002) <br>
138 ALEXANDRA RD - 1.2909986493278949, 103.81773555300782 (136B Alexandra Rd) <br>
142 TIONG BAHRU RD - 1.2860167987575752, 103.82729341802703 (142 Tiong Bahru Rd) <br>
91 ZION RD - 1.2908828260864573, 103.83395829829804 (Singapore 160091) <br>
21 OUTRAM PK - 1.283322012087497, 103.83824591178877 (21 Outram Park) <br>
4 ROCHOR RD - 1.3020007633640034, 103.85495784811125 (4 Rochor Rd) <br>
3 ROCHOR RD - 1.3027627932636705, 103.8552090174156 (3 Rochor Rd) <br>
39A C'WEALTH AVE - 1.298230653234668, 103.80500523774818 (Singapore 140039) <br>
31 DOVER RD - 1.3004422253426429, 103.7838267690774 (Singapore 130031) <br>
89 LOR 2 TOA PAYOH - 1.339307124494977, 103.84692212405666 (89 Lor 2 Toa Payoh) <br>
38 DOVER RD - 1.299062629841912, 103.78219397424145 (Singapore 130038) <br>
17 REDHILL CL - 1.285221270050013, 103.8177752454513 (Singapore 151017) <br>
2 REDHILL CL - 1.284426733178467, 103.81728226876504 (Singapore 151002) <br>
321 CLEMENTI AVE 5 - 1.316592088265206, 103.76633697428424 (Singapore 120321) <br>
7 YUNG KUANG RD - 1.330598729704131, 103.72236826875991 (Singapore 610007) <br>
75 C'WEALTH DR - 1.301213084057412, 103.7959253853315 (Singapore 140075) <br>
167 LOR 1 TOA PAYOH - 1.3336125838574902, 103.84340852344492 (167 Lor 1 Toa Payoh) <br>
6 REDHILL CL - 1.2848011490783848, 103.81712806404295 (Singapore 151006) <br>
10 UPP BOON KENG RD - 1.3142015206597941, 103.8711182156638 (11 Upper Boon Keng Rd) <br>
113 BT MERAH VIEW - 1.286412765091265, 103.82274929208548 (Singapore 150113) <br>
23 KG BAHRU HILL - 1.2772337506414735, 103.82924749848668 (23 Silat Rd) <br>
9 REDHILL CL - 1.2851244369626456, 103.81641495097224 (Singapore 151009) <br>
51 TANGLIN HALT RD - 1.3014265318024425, 103.79770026337631 (49 Tanglin Halt Rd) <br>
5 REDHILL CL - 1.2845567108660667, 103.81779365649693 (Singapore 150005) <br>
89 ZION RD - 1.291552273220678, 103.83345679277531 (Singapore 160089) <br>
16 REDHILL CL - 1.2862001853123681, 103.81705524545117 (Singapore 150016) <br>
18 REDHILL CL - 1.2858596550188974, 103.81779495097402 (Singapore 151018) <br>
21 REDHILL CL - 1.2864675866818298, 103.81766093625994 (Singapore 151021) <br>
14 REDHILL CL - 1.285703347714016, 103.81705224545124 (Singapore 151014) <br>
137 ALEXANDRA RD - 1.2909986493278949, 103.81773555300782 (136B Alexandra Rd) <br>
24 KG BAHRU HILL - 1.277628050622453, 103.82957603406845 (24 Silat Rd) <br>
22 REDHILL CL - 1.286650420795775, 103.8179880270782 (Singapore 151022) <br>
68 DAKOTA CRES - 1.3069593734513223, 103.88779862957955 (68 Dakota Crescent) <br>
11 REDHILL CL - 1.2855060961614753, 103.81621292765057 (Singapore 151011) <br>
7 SELETAR WEST FARMWAY 6 - 1.3964609223019275, 103.87016715218445 (Singapore 798046) <br>
12 REDHILL CL - 1.2861680967484115, 103.8165653353245 (Singapore 151012) <br>
7 REDHILL CL - 1.2849087108418868, 103.81776458655513 (Singapore 151007) <br>
19 KG BAHRU HILL - 1.2771908459602372, 103.82923140523195 (19 Silat Rd) <br>
6 SELETAR WEST FARMWAY 6 - 1.397189888504928, 103.87112093262473 (Singapore 798045)

In [30]:
# Copy the list above into Excel and save it
# Then import that Excel file back to here
flat_coordinates_missing = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'flat_coordinates_missing.csv'))

In [31]:
flat_coordinates_missing

,address,latitude,longitude
0,309 ANG MO KIO AVE 1,1.365517,103.843968
1,247 ANG MO KIO AVE 3,1.368222,103.833653
2,252 ANG MO KIO AVE 4,1.369342,103.834223
3,455 ANG MO KIO AVE 10,1.368841,103.857035
4,4 HILLVIEW AVE,1.366437,103.762224
5,6 HILLVIEW AVE,1.366213,103.762457
6,2 HILLVIEW AVE,1.366203,103.762398
7,33 TAMAN HO SWEE,1.288329,103.831740
8,114 BT MERAH VIEW,1.286387,103.822285
9,18 KG BAHRU HILL,1.276578,103.829977


In [32]:
# Check if there are 137 addresses in 'flat_coordinates_missing'
flat_coordinates_missing.shape

(137, 3)

# 06. Compile A Complete Flat Coordinates List

In [33]:
# 1) Create a deep copy of 'flat_coordinates_onemap'
flat_coordinates_onemap_copy = copy.deepcopy(flat_coordinates_onemap)

In [34]:
# 2) Remove the addresses with missing coordinates in 'flat_coordinates_onemap_copy'
flat_coordinates_onemap_copy = flat_coordinates_onemap_copy.loc[flat_coordinates_onemap_copy['latitude'] != 'Not found']

In [35]:
# Check if there are 9,667 address in 'flat_coordinates_onemap_copy' after removing 137 addresses
flat_coordinates_onemap_copy.shape

(9667, 3)

In [36]:
# 3) Concatnate 'flat_coordinates_onemap_copy' with 'flat_coordinates_missing'
flat_coordinates_complete = pd.concat([flat_coordinates_onemap_copy, flat_coordinates_missing])

In [37]:
flat_coordinates_complete.head()

,address,latitude,longitude
1,216 ANG MO KIO AVE 1,1.36619678831054,103.841505011903
2,211 ANG MO KIO AVE 3,1.369196965617,103.841666636086
3,202 ANG MO KIO AVE 3,1.36844644010937,103.844516260527
4,235 ANG MO KIO AVE 3,1.36682360872342,103.83649123351
5,232 ANG MO KIO AVE 3,1.36834606813391,103.837196046409


In [38]:
flat_coordinates_complete.tail()

,address,latitude,longitude
132,7 SELETAR WEST FARMWAY 6,1.396461,103.870167
133,12 REDHILL CL,1.286168,103.816565
134,7 REDHILL CL,1.284909,103.817765
135,19 KG BAHRU HILL,1.277191,103.829231
136,6 SELETAR WEST FARMWAY 6,1.39719,103.871121


In [39]:
# Check the final number of rows is the same as the initial 'flat_coordinates_onemap' (9,804 rows)
flat_coordinates_complete.shape

(9804, 3)

In [40]:
flat_coordinates_onemap.shape

(9804, 3)

In [41]:
# Check if there is no missing latitude and longitude in 'flat_coordinates_complete'
flat_coordinates_complete.loc[flat_coordinates_complete['latitude'] == 'Not found']['address']

Series([], Name: address, dtype: object)

In [42]:
flat_coordinates_complete.loc[flat_coordinates_complete['longitude'] == 'Not found']['address']

Series([], Name: address, dtype: object)

In [43]:
# 4) Reset the index
flat_coordinates_complete.reset_index(drop = True, inplace = True)

In [44]:
flat_coordinates_complete.head()

,address,latitude,longitude
0,216 ANG MO KIO AVE 1,1.36619678831054,103.841505011903
1,211 ANG MO KIO AVE 3,1.369196965617,103.841666636086
2,202 ANG MO KIO AVE 3,1.36844644010937,103.844516260527
3,235 ANG MO KIO AVE 3,1.36682360872342,103.83649123351
4,232 ANG MO KIO AVE 3,1.36834606813391,103.837196046409


In [45]:
flat_coordinates_complete.tail()

,address,latitude,longitude
9799,7 SELETAR WEST FARMWAY 6,1.396461,103.870167
9800,12 REDHILL CL,1.286168,103.816565
9801,7 REDHILL CL,1.284909,103.817765
9802,19 KG BAHRU HILL,1.277191,103.829231
9803,6 SELETAR WEST FARMWAY 6,1.39719,103.871121


In [46]:
# 5) Export 'flat_coordinates_complete' to Prepared Data folder
flat_coordinates_complete.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'flat_coordinates_complete.csv'))

# 07. Add Flat Coordinates to Main Dataframe

In [47]:
# Main dataframe is 'flat_resale'
flat_resale.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0,309 ANG MO KIO AVE 1
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1,309 ANG MO KIO AVE 1
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2,309 ANG MO KIO AVE 1
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3,309 ANG MO KIO AVE 1
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4,216 ANG MO KIO AVE 1


In [48]:
# Merge 'flat_coordinates_complete' and 'flat_resale' on 'address' column using outer join
flat_resale_merge = flat_resale.merge(flat_coordinates_complete, on = ['address'], how = 'outer', indicator = True)

In [49]:
flat_resale_merge.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address,latitude,longitude,_merge
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0,309 ANG MO KIO AVE 1,1.365517,103.843968,both
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1,309 ANG MO KIO AVE 1,1.365517,103.843968,both
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2,309 ANG MO KIO AVE 1,1.365517,103.843968,both
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3,309 ANG MO KIO AVE 1,1.365517,103.843968,both
4,1990-02-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,8000.0,86,1507,309 ANG MO KIO AVE 1,1.365517,103.843968,both


In [50]:
flat_resale_merge.tail()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address,latitude,longitude,_merge
915366,2023-07-01,YISHUN,5 ROOM,674B,YISHUN AVE 4,10 TO 12,112.0,Improved,2018,715000.0,94,915054,674B YISHUN AVE 4,1.42107254833692,103.843117058752,both
915367,2023-07-01,YISHUN,5 ROOM,674B,YISHUN AVE 4,07 TO 09,112.0,Improved,2018,720000.0,94,915056,674B YISHUN AVE 4,1.42107254833692,103.843117058752,both
915368,2023-08-01,YISHUN,5 ROOM,674B,YISHUN AVE 4,07 TO 09,112.0,Improved,2018,728000.0,94,915068,674B YISHUN AVE 4,1.42107254833692,103.843117058752,both
915369,2023-12-01,YISHUN,5 ROOM,674B,YISHUN AVE 4,10 TO 12,112.0,Improved,2018,735000.0,94,915080,674B YISHUN AVE 4,1.42107254833692,103.843117058752,both
915370,2023-12-01,YISHUN,5 ROOM,674B,YISHUN AVE 4,07 TO 09,112.0,Improved,2018,700000.0,94,915082,674B YISHUN AVE 4,1.42107254833692,103.843117058752,both


In [51]:
# Check the merging rate
flat_resale_merge['_merge'].value_counts(dropna = False)

_merge
both          915371
left_only          0
right_only         0
Name: count, dtype: int64

In [52]:
flat_resale_merge.shape

(915371, 16)

The merge between hdb_resale and hdb_address_coordinates_complete is completed at full rate. 

# 07. Export Data

In [53]:
# Rearrange the dataframe based on 'row_id' in asccending order before exporting data
flat_resale_merge.sort_values('row_id', inplace = True)

In [54]:
flat_resale_merge.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address,latitude,longitude,_merge
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0,309 ANG MO KIO AVE 1,1.365517,103.843968,both
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1,309 ANG MO KIO AVE 1,1.365517,103.843968,both
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2,309 ANG MO KIO AVE 1,1.365517,103.843968,both
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3,309 ANG MO KIO AVE 1,1.365517,103.843968,both
231,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4,216 ANG MO KIO AVE 1,1.36619678831054,103.841505011903,both


In [55]:
flat_resale_merge.tail()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address,latitude,longitude,_merge
527881,2023-12-01,YISHUN,EXECUTIVE,826,YISHUN ST 81,01 TO 03,142.0,Apartment,1988,780000.0,64,915366,826 YISHUN ST 81,1.41417410336287,103.832454191534,both
535072,2023-12-01,YISHUN,EXECUTIVE,834,YISHUN ST 81,01 TO 03,142.0,Apartment,1988,785000.0,64,915367,834 YISHUN ST 81,1.41508806992271,103.833999504365,both
521167,2023-01-01,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,07 TO 09,179.0,Multi Generation,1987,1080000.0,63,915368,666 YISHUN AVE 4,1.41899847213282,103.840568002498,both
521168,2023-12-01,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,01 TO 03,179.0,Multi Generation,1987,975000.0,63,915369,666 YISHUN AVE 4,1.41899847213282,103.840568002498,both
465592,2023-06-01,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,159.0,Multi Generation,1987,935000.0,63,915370,633 YISHUN ST 61,1.41865757414383,103.839842120968,both


In [56]:
# Reset the index
# drop = True, will prevent the old index value be stored as a new column
flat_resale_merge.reset_index(drop = True, inplace = True)

In [57]:
flat_resale_merge.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address,latitude,longitude,_merge
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0,309 ANG MO KIO AVE 1,1.365517,103.843968,both
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1,309 ANG MO KIO AVE 1,1.365517,103.843968,both
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2,309 ANG MO KIO AVE 1,1.365517,103.843968,both
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3,309 ANG MO KIO AVE 1,1.365517,103.843968,both
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4,216 ANG MO KIO AVE 1,1.36619678831054,103.841505011903,both


In [58]:
flat_resale_merge.tail()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address,latitude,longitude,_merge
915366,2023-12-01,YISHUN,EXECUTIVE,826,YISHUN ST 81,01 TO 03,142.0,Apartment,1988,780000.0,64,915366,826 YISHUN ST 81,1.41417410336287,103.832454191534,both
915367,2023-12-01,YISHUN,EXECUTIVE,834,YISHUN ST 81,01 TO 03,142.0,Apartment,1988,785000.0,64,915367,834 YISHUN ST 81,1.41508806992271,103.833999504365,both
915368,2023-01-01,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,07 TO 09,179.0,Multi Generation,1987,1080000.0,63,915368,666 YISHUN AVE 4,1.41899847213282,103.840568002498,both
915369,2023-12-01,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,01 TO 03,179.0,Multi Generation,1987,975000.0,63,915369,666 YISHUN AVE 4,1.41899847213282,103.840568002498,both
915370,2023-06-01,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,04 TO 06,159.0,Multi Generation,1987,935000.0,63,915370,633 YISHUN ST 61,1.41865757414383,103.839842120968,both


In [59]:
# Drop the '_merge' column before exporting
flat_resale_merge.drop(columns = ['_merge'], inplace = True)

In [60]:
flat_resale_merge.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,row_id,address,latitude,longitude
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,9000.0,86,0,309 ANG MO KIO AVE 1,1.365517,103.843968
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,Improved,1977,6000.0,86,1,309 ANG MO KIO AVE 1,1.365517,103.843968
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,Improved,1977,8000.0,86,2,309 ANG MO KIO AVE 1,1.365517,103.843968
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,Improved,1977,6000.0,86,3,309 ANG MO KIO AVE 1,1.365517,103.843968
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,47200.0,85,4,216 ANG MO KIO AVE 1,1.36619678831054,103.841505011903


In [61]:
flat_resale_merge.shape

(915371, 15)

In [62]:
# Export 'flat_resale_merge' into Prepared Data folder
flat_resale_merge.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'flat_resale_all_w_coord.csv'))